#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2019


# Homework 3:   Collaborative Filtering + Matrix Factorization

### 100 points [6% of your final grade]

### Due: Wednesday, March 27, 2019 by 11:59pm

*Goals of this homework:* Understand how collaborative filtering and matrix factorization works. Explore different methods for real-world recommendation senarios.

*Submission instructions (eCampus):* To submit your homework, rename this notebook as `UIN_hw3.ipynb`. For example, my homework submission would be something like `555001234_hw3.ipynb`. Submit this notebook via eCampus (look for the homework 3 assignment there). Your notebook should be completely self-contained, with the results visible in the notebook. We should not have to run any code from the command line, nor should we have to run your code within the notebook (though we reserve the right to do so). So please run all the cells for us, and then submit.

*Late submission policy:* For this homework, you may use as many late days as you like (up to the 5 total allotted to you).

*Collaboration policy:* You are expected to complete each homework independently. Your solution should be written by you without the direct aid or help of anyone else. However, we believe that collaboration and team work are important for facilitating learning, so we encourage you to discuss problems and general problem approaches (but not actual solutions) with your classmates. You may post on Piazza, search StackOverflow, etc. But if you do get help in this way, you must inform us by **filling out the Collaboration Declarations at the bottom of this notebook**. 

*Example: I found helpful code on stackoverflow at https://stackoverflow.com/questions/11764539/writing-fizzbuzz that helped me solve Problem 2.*

The basic rule is that no student should explicitly share a solution with another student (and thereby circumvent the basic learning process), but it is okay to share general approaches, directions, and so on. If you feel like you have an issue that needs clarification, feel free to contact either me or the TA.

# Part 1. Collaborative Filtering (50 points)

In this part, you will implement collaborative filtering on the [MovieLens Latest Datasets](http://files.grouplens.org/datasets/movielens/ml-latest-small-README.html). After removing users who left less than 20 ratings and movies with less than 20 ratings, the provided dataset has only ~1,200 items and ~500 users. You can also check the title and genres of each movie in *movies_info.csv*.

As background, read [Collaborative Filtering Recommender Systems](http://files.grouplens.org/papers/FnT%20CF%20Recsys%20Survey.pdf) and refer to the course slides `week06rec.pdf` for collaborative filtering.

## 1.1 Load the Data

Please download the dataset from Piazza. There are about 65,000 ratings in total. We split the rating data into two set. You will train with 70% of the data (in *train_movie.csv*) and test on the remaining 30% of data (in *test_movie.csv*). Each of train and test files has lines having this format: UserID, MovieID, Rating. 

First you will need to load the data and store it with any structure you like. Please report the numbers of unique users and movies in the dataset. 

In [1]:
# load the data, then print out the number of
# movies and users in each of train and test sets.
import pandas as pd

trainFile = pd.read_csv('train_movie.csv')
testFile = pd.read_csv('test_movie.csv') 

trainFile.columns = ["userID","movieID","rating"]
testFile.columns = ["userID","movieID","rating"]

trainMatrix = trainFile.pivot(index="userID", columns="movieID", values="rating").as_matrix()
testMatrix = testFile.pivot(index="userID", columns="movieID", values="rating").as_matrix()

print("Number of unique users:", trainMatrix.shape[0])
print("Number of unique movie:", trainMatrix.shape[1])


Number of unique users: 541
Number of unique movie: 1211


/Users/anindita/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()
/Users/anindita/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


## 1.2 Implement User-based Collaborative Filtering

In this part, you will implement the basic User–User Collaborative Filtering algorithm introduced in the class. Given the ratings by each user, you are going to try different methods in calculating the similarities between users. You will use equation `(c)` in `week06rec.pdf` (Page 40) to aggregate ratings. Set k = 0.05. Just consider all users as neighbors. That is, while predicting how user $u$ will rate item $i$, $\widehat{C}$ includes all users who have ratings on i in the training set.

*For this memory-based algorithm, you can only rely on the ratings in training set to predict for the testing set. That is, you assume that you don't know any ratings information in the test set except that when you evalaute your model.*

### Cosine Similarity

First, you will try to calculate the similarity between users with cosine similary following the equation on page 16 of [Collaborative Filtering Recommender Systems](http://files.grouplens.org/papers/FnT%20CF%20Recsys%20Survey.pdf). And then you need to predict the rating for each (user, movie) tuple in the test set. *Note: you don't need to subtract user mean baseline from the ratings prior to computing the similarity.*

In [500]:
import numpy as np
import math

#cosine similarity function
def cos_sim(a, b):
    dot_product=0
    norm_a=0
    norm_b=0
    for i in range(0,len(a)):
        if not math.isnan(a[i]) and not math.isnan(b[i]):
            dot_product += a[i]*b[i]
        if not math.isnan(a[i]):
            norm_a+=math.pow(a[i],2)
        if not math.isnan(b[i]):
            norm_b+=math.pow(b[i],2)
    return (dot_product/ (math.sqrt(norm_a) * math.sqrt(norm_b)))

trainCosine=np.zeros((len(trainMatrix),len(trainMatrix)))

#construct cosine similarity matrix
for user in range(0,len(trainMatrix)):
    for u in range(0,len(trainMatrix)):
        trainCosine[user][u]= cos_sim(trainMatrix[user],trainMatrix[u])


In [501]:
userMean = []

#Compute mean user ratings
for u in range(0,trainMatrix.shape[0]):
    sum=0
    count=0
    for i in range(0,trainMatrix.shape[1]):
        if not math.isnan(trainMatrix[u][i]):
            sum+=trainMatrix[u][i]
            count+=1
    userMean.append(sum/count)

k = 0.05
predictedRatings = np.zeros((trainMatrix.shape[0],trainMatrix.shape[1]))

for user in range(0,trainMatrix.shape[0]):
    rMean = userMean[user]
    for item in range(0,trainMatrix.shape[1]):
        val = 0
        for s in range(0,trainMatrix.shape[0]):
            if(trainCosine[user][s]!=1):
                if not math.isnan(trainMatrix[s][item]):
                    val += trainCosine[user][s] * ( trainMatrix[s][item]- userMean[s])
        predictedRatings[user][item]= rMean + k*val


In [502]:
#Function to calculate Root Mean Square Error
def rmse():
    rmse = 0
    count = 0
    squareSum =0
    for user in range(0,testMatrix.shape[0]):
        for item in range(0,testMatrix.shape[1]):
            if not math.isnan(testMatrix[user][item]):
                temp=0
                temp = testMatrix[user][item] - predictedRatings[user][item]
                squareSum+= math.pow(temp,2)
                count+=1
    rmse = math.sqrt(squareSum/count)
    return rmse


#Function to calculate Mean Absolute Error
def mae():
    mae = 0
    count = 0
    sum =0
    for user in range(0,testMatrix.shape[0]):
        for item in range(0,testMatrix.shape[1]):
            if not math.isnan(testMatrix[user][item]):
                temp=0
                temp = abs(testMatrix[user][item] - predictedRatings[user][item])
                sum+=temp
                count+=1
    mae = (sum/count)
    return mae

### Evaluation

You should evaluate your predictions using Mean Absolute Error and Root Mean Squared Error. 

In [503]:
# Report Mean Absolute Error and Root Mean Squared Error for test set
print("Root Mean Squared Error",rmse())   
print("Mean Absolute Error:",mae())

Root Mean Squared Error 0.8991489107420851
Mean Absolute Error: 0.6903937198616394


### Pearson correlation

Then, you will try to calculate the similarity between users with pearson correlation following `week06rec.pdf` (Page 37). And then you need to predict the rating for each (user, movie) tuple in the test set.

In [504]:
#Pearson correlation function
def pearson_correlation(a, b):
    num = 0
    den1 = 0
    den2 = 0
    res = 0
    for i in range(0,trainMatrix.shape[1]):
        if not math.isnan(trainMatrix[a][i]):
            if not math.isnan(trainMatrix[b][i]):
                num_a = (trainMatrix[a][i] - userMean[a])
                num_b = (trainMatrix[b][i] - userMean[b])
                num += num_a * num_b
                den1 += math.pow(num_a,2)
                den2 += math.pow(num_b,2)
    if(den1==0) or (den2==0):
        return 0
    res = num/(math.sqrt(den1*den2))
    return res

trainPearson=np.zeros((len(trainMatrix),len(trainMatrix)))

#construct pearson correlation matrix
for user in range(0,trainMatrix.shape[0]):
    for u in range(0,trainMatrix.shape[0]):
        trainPearson[user][u]= pearson_correlation(user,u)
        

In [505]:
k = 0.05
predictedRatings = np.zeros((trainMatrix.shape[0],trainMatrix.shape[1]))

for user in range(0,trainMatrix.shape[0]):
    rMean = userMean[user]
    for item in range(0,trainMatrix.shape[1]):
        val = 0
        for s in range(0,trainMatrix.shape[0]):
                if not math.isnan(trainMatrix[s][item]):
                    val += trainPearson[user][s] * ( trainMatrix[s][item]- userMean[s])
        predictedRatings[user][item]= rMean + k*val



### Evaluation

You should evaluate your predictions using Mean Absolute Error and Root Mean Squared Error. 

In [506]:
# Report Mean Absolute Error and Root Mean Squared Error for test
print("Root Mean Squared Error",rmse())   
print("Mean Absolute Error:",mae())

Root Mean Squared Error 0.898338623508709
Mean Absolute Error: 0.6917506759527964


### Pearson correlation (varying the threshold)

In [Collaborative Filtering Recommender Systems](http://files.grouplens.org/papers/FnT%20CF%20Recsys%20Survey.pdf), they observe that: 

> Pearson correlation suffers from computing high similarity
between users with few ratings in common. This can be alleviated by setting a threshold on the number of co-rated items
necessary for full agreement (correlation of 1) and scaling the
similarity when the number of co-rated items falls below this
threshold.

So now revise your Pearson to consider a threshold. Try several values and report for one that you think is appropriate.

In [508]:
k = 0.05

def pearson_correlation_w_threshold_Util(a, b, threshold):
    num = 0
    den1 = 0
    den2 = 0
    res = 0
    intersecting_elements = 0
    for i in range(0,trainMatrix.shape[1]):
        if not math.isnan(trainMatrix[a][i]):
            if not math.isnan(trainMatrix[b][i]):
                intersecting_elements +=1
                num_a = (trainMatrix[a][i] - userMean[a])
                num_b = (trainMatrix[b][i] - userMean[b])
                num += num_a * num_b
                den1 += math.pow(num_a,2)
                den2 += math.pow(num_b,2)
    if(den1==0) or (den2==0):
        return 0
    res = (num/(math.sqrt(den1*den2))) 
    return (res* min(intersecting_elements/threshold,1))

#construct pearson correlation matrix
def pearson_correlation_w_threshold(threshold):
    trainPearson=np.zeros((len(trainMatrix),len(trainMatrix)))
    for user in range(0,trainMatrix.shape[0]):
        for u in range(0,trainMatrix.shape[0]):
            trainPearson[user][u]= pearson_correlation_w_threshold_Util(user,u,threshold)
    return trainPearson

def predictRatings(trainPearson):
    predictedRatings = np.zeros((trainMatrix.shape[0],trainMatrix.shape[1]))
    for user in range(0,trainMatrix.shape[0]):
        rMean = userMean[user]
        for item in range(0,trainMatrix.shape[1]):
            val = 0
            for s in range(0,trainMatrix.shape[0]):
                    if not math.isnan(trainMatrix[s][item]):
                        val += trainPearson[user][s] * ( trainMatrix[s][item]- userMean[s])
            predictedRatings[user][item]= rMean + k*val
    return predictedRatings

trainPearson_10=np.zeros((len(trainMatrix),len(trainMatrix)))
trainPearson_15=np.zeros((len(trainMatrix),len(trainMatrix)))
trainPearson_20=np.zeros((len(trainMatrix),len(trainMatrix)))
trainPearson_25=np.zeros((len(trainMatrix),len(trainMatrix)))
trainPearson_30=np.zeros((len(trainMatrix),len(trainMatrix)))
trainPearson_50=np.zeros((len(trainMatrix),len(trainMatrix)))

trainPearson_10 = pearson_correlation_w_threshold(10)
trainPearson_15 = pearson_correlation_w_threshold(15)
trainPearson_20 = pearson_correlation_w_threshold(20)
trainPearson_25 = pearson_correlation_w_threshold(25)
trainPearson_30 = pearson_correlation_w_threshold(30)
trainPearson_50 = pearson_correlation_w_threshold(50)

predictedRatings_10 = np.zeros((trainMatrix.shape[0],trainMatrix.shape[1]))
predictedRatings_15 = np.zeros((trainMatrix.shape[0],trainMatrix.shape[1]))
predictedRatings_20 = np.zeros((trainMatrix.shape[0],trainMatrix.shape[1]))
predictedRatings_25 = np.zeros((trainMatrix.shape[0],trainMatrix.shape[1]))
predictedRatings_30 = np.zeros((trainMatrix.shape[0],trainMatrix.shape[1]))
predictedRatings_50 = np.zeros((trainMatrix.shape[0],trainMatrix.shape[1]))

predictedRatings_10 = predictRatings(trainPearson_10)
predictedRatings_15 = predictRatings(trainPearson_15)
predictedRatings_20 = predictRatings(trainPearson_20)
predictedRatings_25 = predictRatings(trainPearson_25)
predictedRatings_30 = predictRatings(trainPearson_30)
predictedRatings_50 = predictRatings(trainPearson_50)


/Users/anindita/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


In [509]:
def rmse(predictedRatings):
    rmse = 0
    count = 0
    squareSum =0
    for user in range(0,testMatrix.shape[0]):
        for item in range(0,testMatrix.shape[1]):
            if not math.isnan(testMatrix[user][item]):
                temp=0
                temp = testMatrix[user][item] - predictedRatings[user][item]
                squareSum+= math.pow(temp,2)
                count+=1
    rmse = math.sqrt(squareSum/count)
    return rmse


#Function to calculate Mean Absolute Error
def mae(predictedRatings):
    mae = 0
    count = 0
    sum =0
    for user in range(0,testMatrix.shape[0]):
        for item in range(0,testMatrix.shape[1]):
            if not math.isnan(testMatrix[user][item]):
                temp=0
                temp = abs(testMatrix[user][item] - predictedRatings[user][item])
                sum+=temp
                count+=1
    mae = (sum/count)
    return mae

### Evaluation

You should evaluate your predictions using Mean Absolute Error and Root Mean Squared Error. 

In [510]:
# Report Mean Absolute Error and Root Mean Squared Error for test
t = 10 
thresholds = [predictedRatings_10, predictedRatings_15, predictedRatings_20, predictedRatings_25, predictedRatings_30, predictedRatings_50]
for threshold in thresholds:
    print("Threshold", t )
    print("Root Mean Squared Error",rmse(threshold))   
    print("Mean Absolute Error:",mae(threshold))
    if(t==30):
        t+=20
    else:
        t+=5
    print()

Threshold 10
Root Mean Squared Error 0.890270143854433
Mean Absolute Error: 0.6851235437901917

Threshold 15
Root Mean Squared Error 0.8905005640168924
Mean Absolute Error: 0.6853868941141172

Threshold 20
Root Mean Squared Error 0.891857899121167
Mean Absolute Error: 0.6866426283525587

Threshold 25
Root Mean Squared Error 0.8936574572009665
Mean Absolute Error: 0.6883253384684949

Threshold 30
Root Mean Squared Error 0.8956008686671971
Mean Absolute Error: 0.6901420890207776

Threshold 50
Root Mean Squared Error 0.9026670391381861
Mean Absolute Error: 0.6969055995369728



What do you observe? How different are your results for the original Pearson Correlation approach vs. the thresholded version vs. the Cosine Similarity approach? Why do you think this is? *provide a brief (1-2 paragraph) discussion based on these questions.*

**Cosine Similarity** 

RMSE- 0.8991<br>
MAE- 0.6903<br>

**Pearson Correlation**

RMSE- 0.8983<br>
MAE- 0.6917<br>

**Pearson Correlation with Scaling**

a) **Threshold = 10**<br>

   RMSE-0.8902<br>
   MAE- 0.6851<br>
   
b) **Threshold = 25**<br>

   RMSE-0.8936<br>
   MAE- 0.6883<br>
   
c) **Threshold = 50**<br>

   RMSE-0.902<br>
   MAE- 0.696<br>
   
**Pearson correlation with scaling (with threshold = 10) performed the best out of the three approaches.**

**This is because:**

1)Cosine Similarity considers unknown ratings as '0' which implies that the user hated the movie. If the user hasn't watched the movie or just hasn't rated the movie then this might be a misrepresentation. 

2)Pearson Correlation computes statistical correlation between 2 users common ratings to determine their similarity. However, it suffers from computing high similarity between users with few ratings in common.

3)Pearson correlation with scaling limits the neighborhood size which can result in more accurate predictions. Neighbors with low correlation introduce more noise than signal into the process. A threshold value of 10 (amongst 10, 15, 10, 25, 30 and 50) worked best with our dataset (541 users * 1211 movies). The RMSE increased as the threshold value was increased.

# Part 2. MF (20 points)

In class, we introduced how matrix factorization works for recommendation. Now it is your term to implement it. There are different methods to obtain the latent factor matrices **P** and **Q**, like gradient descent, Alternating Least Squares (ALS), and so on. Pick one of them and implement your MF model. *You can refer to tutorials and resources online. Remember our **collaboration policy** and you need to inform us of the resources you refer to.* 

Please report MAE and RMSE of your MF model for the test set. *You are expected to get a lower MAE and RMSE compared with the results in Part 1.*

In [6]:
import numpy as np
def matrix_factorization(R, P, Q, K, steps, alpha, beta):
    Q = Q.T
    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if not math.isnan(R[i][j]):
                    eij = R[i][j] - numpy.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j]) 
    return numpy.dot(P, Q)

In [7]:
N = trainMatrix.shape[0]
M = trainMatrix.shape[1]
K=3
P = numpy.random.normal(scale= float(1/K), size = (N,K))
Q = numpy.random.normal(scale = float(1/K), size = (M,K))

def rmse(predictedRatings):
    rmse = 0
    count = 0
    squareSum =0
    for user in range(0,testMatrix.shape[0]):
        for item in range(0,testMatrix.shape[1]):
            if not math.isnan(testMatrix[user][item]):
                temp=0
                temp = testMatrix[user][item] - predictedRatings[user][item]
                squareSum+= math.pow(temp,2)
                count+=1
    rmse = math.sqrt(squareSum/count)
    return rmse


#Function to calculate Mean Absolute Error
def mae(predictedRatings):
    mae = 0
    count = 0
    sum =0
    for user in range(0,testMatrix.shape[0]):
        for item in range(0,testMatrix.shape[1]):
            if not math.isnan(testMatrix[user][item]):
                temp=0
                temp = abs(testMatrix[user][item] - predictedRatings[user][item])
                sum+=temp
                count+=1
    mae = (sum/count)
    return mae

predictedRatings = np.zeros((N,M))

In [13]:
trainMatrix[:6]

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan,  4., nan, ..., nan, nan, nan]])

In [9]:
import math
predictedRatings = matrix_factorization(trainMatrix,P,Q,K,steps=100, alpha=0.001, beta=0.00145)
print("Root Mean Squared Error",rmse(predictedRatings))   
print("Mean Absolute Error:",mae(predictedRatings))

Root Mean Squared Error 0.8985096127672352
Mean Absolute Error: 0.6893739842834072


In [10]:
predictedRatings

array([[4.67011613, 4.0713135 , 3.62738925, ..., 4.57329504, 4.87069363,
        5.02430444],
       [3.43558993, 3.09701278, 2.87123875, ..., 3.36376298, 3.18105007,
        3.57922887],
       [1.12025432, 1.09521857, 1.01367472, ..., 1.44665716, 1.06270388,
        1.35254493],
       ...,
       [3.03908535, 2.78232532, 2.26772728, ..., 4.33025683, 4.04614443,
        4.21311288],
       [3.7061566 , 3.23923871, 3.0203154 , ..., 3.15437003, 3.34167245,
        3.59359754],
       [3.85930549, 3.44278386, 3.09356335, ..., 4.0057682 , 3.95073561,
        4.24572912]])

In [336]:
predictedRatings = matrix_factorization(trainMatrix,P,Q,K,steps=100, alpha=0.001, beta=0.0015)
print("Root Mean Squared Error",rmse(predictedRatings))   
print("Mean Absolute Error:",mae(predictedRatings))

Root Mean Squared Error 0.893970464473602
Mean Absolute Error: 0.6861664277554949


In [341]:
K=2
P = numpy.random.normal(scale= float(1/K), size = (N,K))
Q = numpy.random.normal(scale = float(1/K), size = (M,K))
predictedRatings = matrix_factorization(trainMatrix,P,Q,K,steps=100, alpha=0.001, beta=0.0015)
print("Root Mean Squared Error",rmse(predictedRatings))   
print("Mean Absolute Error:",mae(predictedRatings))

Root Mean Squared Error 0.8881835092200071
Mean Absolute Error: 0.680724006299843


In [343]:
K=7
P = numpy.random.normal(scale= float(1/K), size = (N,K))
Q = numpy.random.normal(scale = float(1/K), size = (M,K))
predictedRatings = matrix_factorization(trainMatrix,P,Q,K,steps=100, alpha=0.001, beta=0.0045)
print("Root Mean Squared Error",rmse(predictedRatings))   
print("Mean Absolute Error:",mae(predictedRatings))

Root Mean Squared Error 0.8975586386308012
Mean Absolute Error: 0.6877768422182007


In [348]:
K=4
P = numpy.random.normal(scale= float(1/K), size = (N,K))
Q = numpy.random.normal(scale = float(1/K), size = (M,K))
predictedRatings = matrix_factorization(trainMatrix,P,Q,K,steps=100, alpha=0.001, beta=0.00125)
print("Root Mean Squared Error",rmse(predictedRatings))   
print("Mean Absolute Error:",mae(predictedRatings))

Root Mean Squared Error 0.8930243455546183
Mean Absolute Error: 0.682734397973006


In [350]:
K=4
P = numpy.random.normal(scale= float(1/K), size = (N,K))
Q = numpy.random.normal(scale = float(1/K), size = (M,K))
predictedRatings = matrix_factorization(trainMatrix,P,Q,K,steps=100, alpha=0.001, beta=0.0015)
print("Root Mean Squared Error",rmse(predictedRatings))   
print("Mean Absolute Error:",mae(predictedRatings))

Root Mean Squared Error 0.9029429322486798
Mean Absolute Error: 0.6921217263259118


In [353]:
K=4
P = numpy.random.normal(scale= float(1/K), size = (N,K))
Q = numpy.random.normal(scale = float(1/K), size = (M,K))
predictedRatings = matrix_factorization(trainMatrix,P,Q,K,steps=100, alpha=0.001, beta=0.0015)
print("Root Mean Squared Error",rmse(predictedRatings))   
print("Mean Absolute Error:",mae(predictedRatings))

Root Mean Squared Error 0.8906820785559076
Mean Absolute Error: 0.6846433369958664


In [354]:
K=3
P = numpy.random.normal(scale= float(1/K), size = (N,K))
Q = numpy.random.normal(scale = float(1/K), size = (M,K))
predictedRatings = matrix_factorization(trainMatrix,P,Q,K,steps=100, alpha=0.001, beta=0.0015)
print("Root Mean Squared Error",rmse(predictedRatings))   
print("Mean Absolute Error:",mae(predictedRatings))

Root Mean Squared Error 0.9007205331227398
Mean Absolute Error: 0.6882902908325119


Which method did you use to obtain **P** and **Q**? What are the advantages and disadvantages of the method you pick? *provide a brief (1-2 paragraph) discussion based on these questions.*

I used **Stochastic Gradient Descent (SGD)** to obtain latent representations of users and movies.<br>

**Advantages** of SGD over Alternating Least Squares (ALS):<br>
1. SGD is easier to implement and is efficient.
2. SGD iteration is faster compared to ALS because its not as computationally intensive.

**Disadvantages** of SGD:<br>
1. As compared to ALS, SGD needs more iterations to obtain the same level of accuracy.
2. Its performance is sensitive to the choice of the learning rate. 
3. Unlike ALS, parallelization of SGD is challenging.
 

# Part 3. Extension (30 points)

Given your results in the previous parts, can you do better? For this last part you should report on your best attempt at improving MAE and RMSE. Provide code, results, plus a brief discussion on your approach. Hints: You may consider using the title or genres information, trying other algorithms, designing a hybrid system, and so on. *As in the last homework, you can do anything you like to improve MAE and RMSE.*

You will get full marks for this part if you get better results than both of your CF and MF (of course we will also judge whether what you do here is reasonable or not). Additionally, you will get 5 points as bonus if your model performs the best among the whole class.

In [511]:
#Item-item CF
import numpy as np
import math

#cosine similarity function
def cos_sim(a, b):
    dot_product=0
    norm_a=0.0
    norm_b=0.0
    for i in range(0,len(a)):
        if not math.isnan(a[i]) and not math.isnan(b[i]):
            dot_product += a[i]*b[i]
        if not math.isnan(a[i]):
            norm_a+=math.pow(a[i],2)
        if not math.isnan(b[i]):
            norm_b+=math.pow(b[i],2)
    return (dot_product/ (math.sqrt(norm_a) * math.sqrt(norm_b)))

#construct cosine similarity matrix
if(trainMatrix.shape[0]==541):
    trainMatrix = trainMatrix.T #item x user
    
trainCosine=np.zeros((len(trainMatrix),len(trainMatrix)))

for item in range(0,len(trainMatrix)):
    for i in range(0,len(trainMatrix)):
        trainCosine[item][i]= cos_sim(trainMatrix[item],trainMatrix[i])


In [512]:
import pandas as pd
import numpy as np

movies =[]
with open('movies_info.csv') as fin:
    rows = (line.split() for line in fin)
    for row in rows:
        movies.append(row)

movieGenres = []

for movie in range (1,len(movies)):
    movieGenres.append(movies[movie][-1])
    
#set of Genres that we have 
Genres  = [] 
for genre in movieGenres:
    x=genre.split('|')
    for i in x:
        Genres.append(i)

sGenres = set(Genres)

#split the string for genres to get individual genre
for i in range (0, len(movieGenres)):
    movieGenres[i] = movieGenres[i].split('|')
    
vecGenres = []
for m1 in movieGenres:
    temp = {'Action':0, 'Adventure':0,'Animation':0, 'Children':0,'Comedy':0, 'Crime':0, 'Documentary':0, 'Drama':0,'Fantasy':0,'Film-Noir':0,'Horror':0,'IMAX':0,'Musical':0,'Mystery':0,'Romance':0, 'Sci-Fi':0,'Thriller':0,'War':0,'Western':0 }
    for x in range(0,len(m1)):
            temp[m1[x]] = 1
    vecGenres.append(temp)
    
def cos_sim_content(a, b):
    dot_product=0
    norm_a=0.0
    norm_b=0.0
    for genre in sGenres:
        dot_product += a[genre]*b[genre]
        norm_a+=math.pow(a[genre],2)
        norm_b+=math.pow(b[genre],2)
    return (dot_product/ (math.sqrt(norm_a) * math.sqrt(norm_b)))

cosineMatrix = np.zeros((len(movieGenres),len(movieGenres)))

for m in range(0,len(vecGenres)):
    for n in range(0,len(vecGenres)):
        cosineMatrix[m][n]= cos_sim_content(vecGenres[m],vecGenres[n])

In [513]:
#Take average of both the cosine matrices that I have. 
# cosineMatrix - Content based using movie genres
# trainCosine - Item-Item CF for user*item matrix
avg_cosScore=(trainCosine+cosineMatrix)/2
avg_cosScore

array([[1.        , 0.5527275 , 0.27350603, ..., 0.08417788, 0.30855061,
        0.05711708],
       [0.5527275 , 1.        , 0.08620977, ..., 0.08627652, 0.39101573,
        0.12642038],
       [0.27350603, 0.08620977, 1.        , ..., 0.0265849 , 0.0048825 ,
        0.02831792],
       ...,
       [0.08417788, 0.08627652, 0.0265849 , ..., 1.        , 0.44122603,
        0.56838204],
       [0.30855061, 0.39101573, 0.0048825 , ..., 0.44122603, 1.        ,
        0.60343348],
       [0.05711708, 0.12642038, 0.02831792, ..., 0.56838204, 0.60343348,
        1.        ]])

In [514]:
itemMean = []
#Compute mean item ratings
for i in range(0,trainMatrix.shape[0]):
    sum=0
    count=0
    for u in range(0,trainMatrix.shape[1]):
        if not math.isnan(trainMatrix[i][u]):
            sum+=trainMatrix[i][u]
            count+=1
    itemMean.append(sum/count)

In [515]:
#print(trainMatrix.shape)
predictedRatings = np.zeros((trainMatrix.shape[0],trainMatrix.shape[1]))

for item in range(0,trainMatrix.shape[0]):
    iMean = itemMean[item]
    for user in range(0,trainMatrix.shape[1]):
        val = 0
        tot_cos = 0
        for s in range(0,trainMatrix.shape[0]):
            if(trainCosine[item][s]!=1):
                if not math.isnan(trainMatrix[s][user]):
                    val += avg_cosScore[item][s] * (trainMatrix[s][user]- itemMean[s])
                    tot_cos += abs(avg_cosScore[item][s])
        predictedRatings[item][user]= iMean + float(val/tot_cos)


In [516]:
predictedRatings

array([[4.84263715, 3.74933971, 1.49196837, ..., 3.52114138, 3.74995444,
        4.12560946],
       [4.29428234, 3.04341855, 1.51836043, ..., 3.00615714, 3.10030957,
        3.56645185],
       [4.12277257, 3.4028694 , 0.22452412, ..., 2.76307598, 3.0103708 ,
        3.46175284],
       ...,
       [4.65313344, 3.32868516, 3.29195196, ..., 3.7972527 , 3.62217315,
        4.12316432],
       [4.68972163, 3.52263614, 3.17016203, ..., 3.67237632, 3.60565048,
        4.04937533],
       [5.00293062, 3.91622908, 3.57282585, ..., 4.15057594, 4.04216046,
        4.41069337]])

In [517]:
if(testMatrix.shape[0] == 541):
    testMatrix = testMatrix.T

def rmse():
    rmse = 0
    count = 0
    squareSum =0
    for user in range(0,testMatrix.shape[0]):
        for item in range(0,testMatrix.shape[1]):
            if not math.isnan(testMatrix[user][item]):
                temp=0
                temp = testMatrix[user][item] - predictedRatings[user][item]
                squareSum+= math.pow(temp,2)
                count+=1
    rmse = math.sqrt(squareSum/count)
    return rmse


#Function to calculate Mean Absolute Error
def mae():
    mae = 0
    count = 0
    sum =0
    for user in range(0,testMatrix.shape[0]):
        for item in range(0,testMatrix.shape[1]):
            if not math.isnan(testMatrix[user][item]):
                temp=0
                temp = abs(testMatrix[user][item] - predictedRatings[user][item])
                sum+=temp
                count+=1
    mae = (sum/count)
    return mae

print("Root Mean Squared Error",rmse())   
print("Mean Absolute Error:",mae())

Root Mean Squared Error 0.8689594285741693
Mean Absolute Error: 0.6681990167907204


Please explain what you do to improve the recommendation in 1-2 paragraphs.

Until now, we used user-user collaborative filtering which suffers from:
1. Data Sparsity: In case of large number of items, number of items a user has rated reduces to a tiny percentage making the correlation coefficient less reliable
2. User profiles change quickly and the entire system model had to be recomputed which is both time and computationally expensive

So, I decided to use item-item collaborative filtering. This method is quite stable in itself as compared to User based collaborative filtering because the average item has a lot more ratings than the average user. So an individual rating doesn’t impact as much. However, popularity of a movie can also easily influence collaborative recommendations. Therfore, introducing a content based filtering approach over metadata (such as movie genre) could be an useful feature to consider.

I simply averaged the cosine similarities to ensemble the results from the content and the collaborative models and then used a simple weighted average approach to predict ratings. I was able to achieve an RMSE of 0.8690 and MAE of 0.6682. 

By ensembling item-item CF and content-based results I was able to outperform user-user CF (with cosine similarity, pearson correlation and pearson correlation with scaling approaches). 


## Collaboration declarations

*If you collaborated with anyone (see Collaboration policy at the top of this homework), you can put your collaboration declarations here.*

In [ ]:
For MF, I found helpful suggestions on http://www.quuxlabs.com/blog/2010/09/matrix-factorization-a-simple-tutorial-and-implementation-in-python/